In [1]:
from datetime import datetime, timedelta
import pandas as pd

from wellplan.core import TeamPool, Task
from wellplan.services import (
    TeamManager,
    FileProductionProfile,
    ArpsDeclineProductionProfile,
    NPV,
    BaseCapex,
    BaseOpex,
    ClusterRandomRiskStrategy,
    DistanceTeamMovement,
    OilConstraint,
    CapexConstraint
)
from wellplan.services.visualization import GraphVisualizer, GantVisualizer, CapturePlot
from wellplan.data.file.loader import ExcelWellLoader
from wellplan.data.file.saver import ExcelPlanSaver
from wellplan.builder import PlanBuilder

In [2]:
loader = ExcelWellLoader(
    file_path=r"external/Ввод ЭБ 1+11 Обезлич_v3.xlsx",
)
wells = loader.load()
production_profile = FileProductionProfile(folder_path='external/profiles')
# production_profile = ArpsDeclineProductionProfile()
coordinates = pd.read_excel(
    r"external/База перспективного ПФ (30.01.2025)_fixed.xlsm",
    header=0,
    names=["cluster", "x", "y", "z"],
)
movement = DistanceTeamMovement.from_dicts(coordinates.to_dict(orient="records"))

2025-04-11 16:10:28.737 | INFO     | wellplan.data.file.profile_loader:_load_cache:63 - Cache file for /home/ruslan/store/work_liga/Projects/well-plan-optimization/notebooks/external/profiles is detected
2025-04-11 16:10:28.759 | INFO     | wellplan.data.file.profile_loader:load:47 - Processing folder /home/ruslan/store/work_liga/Projects/well-plan-optimization/notebooks/external/profiles:
            0 added,
            0 removed,
            0 modified files


In [8]:
team_pool = TeamPool()
team_pool.add_teams(['ГС'], num_teams=7)
team_pool.add_teams(['грп'], num_teams=7)


capex = BaseCapex(
    build_cost_per_metr={
        "ГС+ГРП": 25300,
        "ННС+ГРП": 12900,
        "МЗС": 27300,
        "МЗС+ГРП": 28300,
        "ГС": 23300,
    },
    equipment_cost=2500000,
)

opex = BaseOpex(
    oil_cost_per_tone=109.9,
    water_cost_per_tone=48.6,
    repair_per_year=3093900,
    maintain_per_year=2336200,
)


npv = NPV(
    oil_price_per_tone=13896,
    project_start_date=datetime.now(),
    capex_cost=capex,
    opex_cost=opex,
    discount_rate=0.125,
)

constraints = [
    CapexConstraint([{
        "year": 2025,
        'value': 500000000
    }]),
    OilConstraint([{
        "year": 2025,
        'value': 100000
    },
    {
        "year": 2026,
        'value': 50000
    }
        ]),
]

builder = PlanBuilder(
    start=datetime.now(),
    end=datetime.now() + timedelta(days=365 * 25),
    cost_function=npv,
    production_profile=production_profile,
    constraints=constraints,
)


In [9]:
limits = {
    2025: {Task.from_code("DRILLING"): 2, Task.from_code("GTM"): 2},
    2026: {Task.from_code("DRILLING"): 5, Task.from_code("GTM"): 7},
    2027: {Task.from_code("DRILLING"): 3, Task.from_code("GTM"): 3},
}


In [ ]:
plan = builder.compile(
        wells,
        manager=TeamManager(
            team_pool=team_pool,
            movement=movement,
            # limits=limits,
        ),
        risk_strategy=ClusterRandomRiskStrategy(trigger_chance=0.0),
    )
print(plan)
plan.total_profit()

In [ ]:
saver = ExcelPlanSaver('plan.xlsx')
saver.save(plan)

In [ ]:
viz = GraphVisualizer(figsize=(50, 10))
viz.render(plan)

In [ ]:
viz = GantVisualizer(figsize=(30, 5))
viz.render_by_teams(plan)

In [ ]:
viz = GantVisualizer(figsize=(10, 30))
viz.render(plan)

In [ ]:
plans = []
for i in range(5):
    plan = builder.compile(
        wells,
        manager=TeamManager(
            team_pool=team_pool,
        ),
        risk_strategy=ClusterRandomRiskStrategy(trigger_chance=0.1),
    )
    plans.append(plan)

In [ ]:
viz = GraphVisualizer(figsize=(50, 10))
viz.render_multiple(plans)

In [ ]:
initial_plan = builder.compile(
        wells,
        manager=TeamManager(
            team_pool=team_pool,
            movement=movement,
        ),
        risk_strategy=ClusterRandomRiskStrategy(trigger_chance=0.0),
        keep_order=True
    )
optimized_plan = builder.compile(
        wells,
        manager=TeamManager(
            team_pool=team_pool,
            movement=movement,
        ),
        risk_strategy=ClusterRandomRiskStrategy(trigger_chance=0.0),
        keep_order=False
    )
viz = GraphVisualizer(figsize=(50, 30), title='Аккумулированный NPV')
viz.render_multiple([initial_plan, optimized_plan])

In [ ]:
optimized_plan.total_profit() - initial_plan.total_profit()

In [ ]:
viz = GantVisualizer(figsize=(30, 12))
viz.render_by_teams(initial_plan)